In [ ]:
!pip install transformers
!pip install -U spacy
!python3 -m spacy download en_core_web_sm
!pip install wordcloud
!pip install datasets
!pip install -q -U google-generativeai

In [ ]:
# 라이브러리 임포트
import os
import random
import re
import time
import datetime
from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import spacy

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from wordcloud import WordCloud

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, BertConfig

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# 하이퍼파라미터
args = {
    "train_path" : "",      # train 데이터 경로
    "test_path" : "",       # test 데이터 경로
    "submit_path" : "",     # submit 파일 경로
    "max_len" : 128,
    "batch_size" : 32,
    "num_labels" : 2,
    "epochs" : 1,
    "lr" : 2e-5, 
    "eps" : 1e-8,
    "seed_val" : 42     # 절대 수정하지 마세요.
}

In [ ]:
# 랜덤시드 고정하기
seed = args["seed_val"]
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available() : 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# 디바이스 선택
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# **1. 데이터**

## **(1) 데이터 불러오기**

In [ ]:
train = pd.read_csv(args["train_path"])
test = pd.read_csv(args["test_path"])

## **(2) 데이터 시각화 및 분석**

### **1) 데이터 확인**

In [ ]:
train.head()

In [ ]:
test.head()

### **2) 결괏값 분석**

In [ ]:
train.describe()

In [ ]:
test.describe()

### **3) 결측치**

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# (1) 결측값(NaN)을 공백 문자열로 채움
train.fillna(' ', inplace=True)
test.fillna(' ', inplace=True)

In [ ]:
print(f'train shape : {train.shape}')
print(f'test shape : {test.shape}')

### **4) 예측 클래스(fraudulent) 분포**

In [ ]:
sns.countplot(x='fraudulent',data=train)
plt.title('Distribution of the "fraudulent" attribute')
plt.xlabel('Is the job posting fake? (0 - No, 1 - Yes)')
plt.ylabel('Number of job postings')
plt.show()

### **5) 열 항목값들과 예측 클래스(fraudulent) 사이의 관계 확인**

In [ ]:
# 대상 계층과 고용 유형 간에 관계가 있는지 확인
plt.figure(figsize=(10, 6))
sns.countplot(data=train, x='employment_type', hue='fraudulent')
plt.title('Count of Fraudulent Cases by Employment Type')
plt.xlabel('Employment Type')
plt.ylabel('Count')
plt.legend(title='Fraudulent')
plt.show()

In [ ]:
# 대상 클래스와 필수 경험 간에 관계가 있는지 확인
plt.figure(figsize=(10, 6))
sns.countplot(data=train, x='required_experience', hue='fraudulent')
plt.title('Count of Fraudulent Cases by Required Experience')
plt.xlabel('Required Experience')
plt.ylabel('Count')
plt.legend(title='Fraudulent')
plt.show()

In [ ]:
# 대상 클래스와 필수 교육 사이에 관계가 있는지 확인
plt.figure(figsize=(30, 18))
sns.countplot(data=train, x='required_education', hue='fraudulent')
plt.title('Count of Fraudulent Cases by Required Education')
plt.xlabel('Required Education')
plt.ylabel('Count')
plt.legend(title='Fraudulent')
plt.show()

## **(3) 데이터 전처리**

### **1) 기본 데이터 전처리**

#### **Concatenate**
- 여러 텍스트 열을 하나의 text 열로 합치고, 그 외의 불필요한 열들을 삭제하여 데이터를 정리하고 준비

In [ ]:
def concatenate(data):
    """
    주어진 데이터프레임에서 텍스트 열을 합치고 필요없는 열을 삭제하는 함수입니다.

    Args:
        data (pd.DataFrame): 합칠 데이터프레임

    Returns:
        pd.DataFrame: 텍스트 열이 합쳐진 후 필요없는 열이 삭제된 데이터프레임
    """
    # (2-1) 데이터프레임의 각 텍스트 열을 하나의 텍스트 열로 합칩니다.
    data['text'] = 
    
    # (2-2) 삭제할 열 목록 정의
    delete_list = 
    
    # (2-3) 데이터프레임에서 삭제할 열을 삭제합니다. (drop 함수 이용)
    data = 
    
    return data

### **텍스트 데이터 정제**
1. 줄 바꿈, 캐리지 리턴, 탭 제거 :

    - \n, \r, \t와 같은 특수 문자는 일반적으로 텍스트 처리 시 불필요한 경우가 많습니다. 

    - 이를 제거하여 텍스트를 깔끔하게 만듭니다.

2. 숫자 제거 :

    - 텍스트 데이터에서 숫자는 주로 의미를 가지지 않는 경우가 많습니다. 

    - 따라서 숫자를 제거하여 텍스트의 일관성을 유지합니다.

3. 특수 문자 공백으로 대체 :

    - [/(){}\[\]\|@,;.:-]와 같은 특수 문자는 자연어 처리 모델에 있어서는 주로 불필요한 경우가 많습니다. 

    - 이를 공백으로 대체하여 단어들이 제대로 분리되도록 합니다.

4. 텍스트를 소문자로 변환 :

    - 텍스트의 일관성을 유지하기 위해 모든 문자를 소문자로 변환합니다. 
    
    - 대소문자의 구분은 자연어 처리 모델에서는 의미가 달라질 수 있기 때문에 일반적으로 소문자로 통일합니다.

5. 불필요한 공백 제거 :

    - 전처리 과정에서 생길 수 있는 연속된 공백을 하나의 공백으로 줄입니다.

In [ ]:
def text_preprocessing(document):
    """
    주어진 텍스트 데이터를 전처리하는 함수입니다.

    Args:
        document (pd.Series): 전처리할 텍스트 데이터가 포함된 pandas Series

    Returns:
        pd.Series: 전처리가 완료된 텍스트 데이터가 포함된 pandas Series
    """
    # (3-1) 줄 바꿈(\n), 캐리지 리턴(\r) 및 탭(\t)을 제거
    document = 
    document = 
    document = 
    
    # (3-2) 숫자를 제거
    document = 
    
    # (3-3) 특수 문자를 공백으로 대체
    document = 
    
    # (3-4) 텍스트를 소문자로 변환
    document = 
    
    # (3-5) 불필요한 공백 제거
    document = 
    
    return document

### **불용어 제거**
- 자연어 처리에서 처리하지 않거나 제거하는 단어들을 말합니다. 

- 이러한 단어들은 문장에서 주로 등장하지만 분석에 있어서는 일반적으로 정보를 제공하지 않거나 미미한 역할을 하는 경우가 많습니다. 

- 따라서 이러한 불용어들을 제거함으로써 텍스트 데이터의 처리나 분석 성능을 개선할 수 있습니다.

- 보통의 불용어 목록에는 다음과 같은 단어들이 포함될 수 있습니다:

    - **영어**: "a", "an", "the", "in", "on", "at", "to" 등

    - **한국어**: "을", "를", "이", "가", "은", "는", "에서", "에서는" 등

- 이러한 단어들은 문법적으로 필수적이거나 의미를 구체적으로 전달하지 않는 경우가 많습니다. 

- 예를 들어, "the"는 문장에서 주로 등장하지만 분석에 있어서는 거의 항상 무시할 수 있는 단어입니다. 

- 또한, 이러한 단어들이 많이 사용될수록 문장의 길이는 더 길어지며, 분석에 방해가 될 수 있습니다.

In [ ]:
def remove_stopwords(document, nlp):
    """
    주어진 텍스트 데이터에서 불용어를 제거하는 함수입니다.

    Args:
        document (pd.DataFrame): 전처리할 텍스트 데이터가 있는 pandas DataFrame. 'text' 열이 필요합니다.
        nlp (spacy.lang.en.English): Spacy에서 로드한 영어 언어 모델 객체.

    Returns:
        pd.DataFrame: 불용어가 제거된 텍스트 데이터가 포함된 pandas DataFrame.
    """
    # (4) 불용어 제거
    # 주어진 document DataFrame의 'text' 열에서 각 문서에 대해 처리를 수행합니다.
    # 각 문서는 공백을 기준으로 단어로 분할(split)하고, 불용어가 아닌 단어들로 구성된 리스트를 생성합니다.
    # 이 리스트는 다시 공백을 사이에 두고 join하여 하나의 문자열로 만듭니다.
    # 이후 이 문자열을 'text' 열에 대입하여 각 문서의 전처리 결과를 업데이트합니다.
    document = 
    
    return document

### **어간 추출(Stemming) 및 표제어 추출(Lemmatization)**
- 두 기법 모두 자연어 처리에서 텍스트 전처리 과정 중에 단어의 원형을 찾는 데 사용됩니다.

- 일반적으로 표제어 추출은 어간 추출보다 정교하고 문맥을 잘 이해하는 방식이기 때문에, 자연어 처리에서 더 많이 사용됩니다.<br><br>

- **어간 추출** : 단순한 접미사 제거 방식으로 단어의 어간을 추출합니다. 결과는 사전에 있는 단어가 아닐 수 있습니다.

- **표제어 추출** : 문법적, 의미적으로 올바른 표준 형태의 단어를 추출합니다. 결과는 사전에 등재된 실제 단어입니다.<br><br>

1. **어간 추출 (Stemming)** : 

    - 어간 추출은 단어의 어간(stem)만 남기고 접사(suffix)를 제거하여 단어의 기본 형태를 찾는 과정입니다.

    - 예를 들어, "running", "runs", "runner"는 모두 "run"이라는 어간을 가지고 있습니다. 따라서 어간 추출은 이들을 모두 "run"으로 축소합니다.

    - 어간 추출은 단순히 규칙 기반의 접미사 제거 방식이기 때문에, 결과로 나오는 단어는 실제 사전에 있는 단어일 필요는 없고, 때로는 일반적이지 않은 형태일 수 있습니다.<br><br>


2. **표제어 추출 (Lemmatization)** :

    - 표제어 추출은 단어의 표준 형태인 표제어(lemma)를 찾는 과정입니다.

    - 표제어는 사전에 등재된 실제 단어입니다. 예를 들어, "running"의 표제어는 "run", "better"의 표제어는 "good"입니다.

    - 표제어 추출은 단어의 문맥을 고려하고, 형태소 분석(morphological analysis)을 수행하여 단어의 형태와 문법적 의미를 이해한 후에 추출합니다.

    - 따라서 표제어 추출은 언어적, 문법적 규칙을 적용하여 단어를 정규화하는 과정입니다.

In [ ]:
# 7~10분 소요
def lemmatize_text(document, nlp):
    """
    주어진 텍스트 데이터에 대해 Spacy 모델을 사용하여 각 단어의 기본형을 추출하는 함수입니다.

    Args:
        document (pd.Series): 전처리할 텍스트 데이터가 포함된 pandas Series.
        nlp (spacy.lang.en.English): Spacy에서 로드한 영어 언어 모델 객체.

    Returns:
        pd.Series: 전처리가 완료된 텍스트 데이터가 포함된 pandas Series.
    """
    # (5) 표제어 추출
    # Spacy 모델인 nlp를 사용하여 텍스트를 토큰화합니다.
    # 각 토큰의 기본형(lemma)을 추출하여 리스트로 변환합니다.
    # 추출된 기본형들을 공백을 사이에 두고 하나의 문자열로 결합합니다.
    # 최종적으로 각 텍스트 데이터의 단어들이 기본형으로 변환된 후의 문자열이 포함된 pandas Series를 반환합니다.
    document = 

    return document


#### **종합**
- concatenate -> 텍스트 데이터 정제 -> 불용어 제거 -> 표제어 추출

In [ ]:
def preprocessing(data):
    """
    주어진 데이터프레임을 전처리하는 함수입니다.

    Args:
        data (pd.DataFrame): 전처리할 데이터가 포함된 pandas DataFrame. 필수 컬럼: 'title', 'department', 'company_profile',
                             'description', 'requirements', 'benefits'.

    Returns:
        pd.DataFrame: 전처리된 데이터가 포함된 pandas DataFrame.
    """
    # 영어 언어 모델 'en_core_web_sm'을 로드합니다.
    nlp = spacy.load("en_core_web_sm")
    
    # (6-1) 데이터프레임의 각 텍스트 열을 하나의 텍스트 열로 합칩니다.
    data = 
    
    # (6-2) 텍스트 데이터 전처리를 수행합니다.
    data['text'] = 
    
    # (6-3) 불용어를 제거합니다.
    data['text'] = 
    
    # (6-4) 각 단어를 기본형으로 변환하여 텍스트를 정제합니다.
    data['text'] = 
    
    return data

In [ ]:
# 10 ~ 15분 소요
train = preprocessing(train)
test = preprocessing(test)

In [ ]:
train

In [ ]:
test

### **※ 워드 클라우드 (Word Cloud)**
- 텍스트 데이터에서 가장 빈도가 높은 단어들을 시각적으로 표현하여, 단어의 중요도를 직관적으로 파악할 수 있게 해주는 시각화 기법입니다. 

- 주어진 텍스트 데이터에서 각 단어의 출현 빈도를 분석하여, 빈도가 높은 단어일수록 크고, 낮은 단어일수록 작게 표현하는 형태로 만듭니다.

In [ ]:
# 실제 잡포스팅 텍스트를 하나의 문자열로 결합
non_fraudulent_text = ' '.join(text for text in train[train['fraudulent'] == 0]['text'])

# 가짜 잡포스팅 텍스트를 하나의 문자열로 결합
fraudulent_text = ' '.join(text for text in train[train['fraudulent'] == 1]['text'])

# 실제 잡포스팅 텍스트의 워드클라우드 생성
wordcloud_non_fraudulent = WordCloud(width=800, height=400, background_color='black').generate(non_fraudulent_text)

# 가짜 잡포스팅 텍스트의 워드클라우드 생성
wordcloud_fraudulent = WordCloud(width=800, height=400, background_color='black').generate(fraudulent_text)

# 두 개의 서브플롯을 가지는 Figure 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# 첫 번째 서브플롯에 진짜 잡포스팅 워드클라우드 표시
ax1.imshow(wordcloud_non_fraudulent, interpolation='bilinear')
ax1.axis('off')  # 축 숨기기
ax1.set_title('Non-Fraudulent Job Postings')

# 두 번째 서브플롯에 가짜 잡포스팅 워드클라우드 표시
ax2.imshow(wordcloud_fraudulent, interpolation='bilinear')
ax2.axis('off')  # 축 숨기기
ax2.set_title('Fraudulent Job Postings')

# 워드클라우드 시각화 표시
plt.show()

### **2) BERT 모델 입력을 위한 데이터 전처리**

#### **스페셜 토큰 붙이기**
- BERT(Bidirectional Encoder Representations from Transformers)에서는 특별한 몇 가지 토큰이 있습니다. 

- 이들 특별 토큰은 BERT 모델이 입력 시퀀스의 시작과 끝, 패딩, 문장 구분 등을 인식하도록 돕습니다.<br><br>

- 주요 스페셜 토큰은 다음과 같습니다:
    1. **[CLS] 토큰**
        - Classification을 위한 토큰입니다.

        - BERT 모델은 입력 시퀀스의 처음에 [CLS] 토큰을 추가하고, 이 위치의 은닉 상태를 활용하여 분류 작업 등에 사용합니다.
        
    2. **[SEP] 토큰**
        - 문장 구분을 위한 토큰입니다. 

        - BERT 모델은 두 개의 문장을 구분할 때 [SEP] 토큰을 사용하여 두 문장을 구분합니다. 또한, 입력 시퀀스의 끝에도 [SEP] 토큰이 추가됩니다.

In [ ]:
def add_special_token(document):
    """
    입력된 문서(문장 리스트)에 BERT 특수 토큰 [CLS]와 [SEP]을 추가하는 함수입니다.

    Args:
        document (list): 문장들이 포함된 리스트.

    Returns:
        list: BERT 특수 토큰 [CLS]와 [SEP]이 추가된 문장들의 리스트.
    """
    added = ["[CLS]" + str(sentence) + "[SEP]" for sentence in document]
    
    return added

#### **토큰화(Tokenization)**
- 토큰화는 텍스트를 작은 단위로 나누는 과정을 말합니다. 

- 이 작은 단위는 일반적으로 <U>토큰(Token)</U>이라고 부릅니다. 

- 토큰은 문장을 구성하는 최소 단위로, 예를 들어 단어, 숫자, 문장부호 등이 될 수 있습니다. 

- 토큰화는 텍스트 데이터를 모델이 이해할 수 있는 형태로 변환하는 첫 단계입니다.<br><br>

- 예시:
    - 입력 텍스트: "Hello, how are you?"
    
    - 토큰화 결과: ["Hello", ",", "how", "are", "you", "?"]<br><br>

- 자연어 처리 모델은 보통 토큰화된 단어들을 입력으로 받아 처리합니다. 

- 토큰화는 단순히 공백을 기준으로 나누는 것 이상의 복잡한 규칙을 적용할 수 있습니다. 

- 예를 들어, 문장부호 처리, 대소문자 통일, 특수 문자 처리 등이 포함될 수 있습니다.


In [ ]:
def tokenization(data):
    """
    입력된 데이터를 BERT 토크나이저를 사용하여 토큰화하고, 토큰을 정수 인덱스로 변환하는 함수입니다.

    Args:
        data (list): 토큰화할 문장들이 포함된 리스트.

    Returns:
        list: 각 문장을 정수 인덱스로 변환한 결과 리스트(ids).
    """
    # (7-1) BERT 토크나이저 초기화 (bert-base-multilingual-cased 모델 사용)
    tokenizer = 
    
    # (7-2) 각 문장을 토큰화
    tokenized = 
    
    # (7-3) 토큰을 정수 인덱스로 변환
    ids = 
    
    return ids


#### **패딩(padding)**
- 패딩은 입력 데이터의 길이를 맞추는 과정을 말합니다. 

- 특히 자연어 처리에서는 모든 입력 데이터가 동일한 길이를 가져야 하는 경우가 많습니다. 

- 이를 위해 길이가 짧은 문장에는 패딩 토큰을 추가하여 전체 데이터셋의 입력 길이를 통일시키는 작업을 수행합니다.<br><br>

- 패딩은 일반적으로 0이나 다른 특정한 값으로 채워진 토큰을 추가하는 방식으로 이루어집니다. 

- 이러한 패딩 과정은 모델이 배치(batch) 단위로 데이터를 처리할 때 효율적으로 동작할 수 있도록 합니다.<br><br>

- 예시:

    - 입력 문장들: ["I", "am", "fine"], ["How", "are", "you", "?"]

    - 패딩 후 결과: ["I", "am", "fine", 0], ["How", "are", "you", "?"]
    
- 여기서 패딩을 추가할 때 주의할 점은 최대 길이를 정하고 그 이하의 문장에 패딩을 추가하는 것입니다. 너무 긴 문장은 잘라내거나 특정 길이로 자를 수도 있습니다.<br><br>

- 토큰화와 패딩은 모델이 텍스트 데이터를 이해하고 처리할 수 있도록 데이터를 준비하는 중요한 단계입니다. 
- 이 두 가지 과정은 대부분의 자연어 처리 모델에서 필수적으로 적용됩니다.

In [ ]:
def padding(ids, args):
    """
    입력된 정수 시퀀스(ids)들을 패딩하여 모든 시퀀스가 동일한 길이가 되도록 만드는 함수입니다.

    Args:
        ids (list of list): 패딩할 정수 시퀀스들이 포함된 리스트.
        args (dict): 패딩에 사용할 매개변수들을 포함한 사전.

    Returns:
        list of list: 패딩된 정수 시퀀스들의 리스트.
    """
    ids = pad_sequences(ids, maxlen=args["max_len"], dtype="long", truncating='post', padding='post').tolist()
    
    return ids


#### **어텐션 마스팅**
- 어텐션 마스킹(Attention Masking)은 자연어 처리에서 Transformer 모델과 같은 어텐션 기반 모델에서 사용되는 중요한 개념입니다.

- 이는 모델이 입력 시퀀스에서 각 토큰 간의 관계를 학습할 때, 불필요한 관계를 제거하거나 유용한 관계를 강조하는 메커니즘입니다.

In [ ]:
def attention_mask(ids):
    """
    입력된 정수 시퀀스들에 대한 어텐션 마스크를 생성하는 함수입니다.

    Args:
        ids (list of list): 정수 시퀀스들이 포함된 리스트.

    Returns:
        list of list: 어텐션 마스크 리스트. 토큰이 있는 부분은 1, 없는 부분은 0으로 설정됩니다.
    """
    masks = []
    
    for id in ids:
        # (8-1) 토큰이 있는 부분은 1, 없는 부분은 0으로 설정
        mask = 
        
        # (8-2) 생성된 마스크를 마스크 리스트에 추가
        
        
    return masks


#### **종합**
- 스페셜 토큰 추가 -> 분절화(토큰화) -> 패딩 -> 마스킹 과정

In [ ]:
def bert_preprocessing(data, args):
    """
    BERT 입력을 준비하기 위한 전처리 함수입니다.

    Args:
        data (DataFrame): 전처리할 데이터가 포함된 pandas DataFrame.
        args (dict): 전처리 과정에서 사용할 인자들이 포함된 딕셔너리.

    Returns:
        DataFrame: BERT 입력을 준비한 후의 데이터프레임.
    """
    # (9-1) 특수 토큰 추가 함수 호출
    document = 
    
    # (9-2) 토큰화 함수 호출하여 텍스트를 토큰 ID로 변환
    ids = 
    
    # (9-3) 패딩 함수 호출하여 모든 시퀀스의 길이를 일치시킴
    ids = 
    
    # (9-4) 어텐션 마스크 생성 함수 호출
    masks = 
    
    # (9-1) 데이터프레임에 토큰 ID 및 어텐션 마스크 열 추가
    data['id'] = 
    data['mask'] = 
    
    # 메모리에서 불필요한 변수 삭제
    del document, ids, masks
    
    # 처리된 데이터 반환
    return data


In [ ]:
train = bert_preprocessing(train, args)
test = bert_preprocessing(test, args)

In [ ]:
train

In [ ]:
test

# **2. 데이터셋과 데이터로더**
참고 : https://pytorch.org/docs/stable/data.html#torch.utils.data.TensorDataset

In [ ]:
# train 데이터와 validation 데이터로 분리
train, valid = train_test_split(train, test_size=0.1)

# TensorDataset을 사용하여 데이터셋 생성
train_dataset = TensorDataset(torch.tensor(train['id'].tolist()), 
                              torch.tensor(train['mask'].tolist()), 
                              torch.tensor(train['fraudulent'].tolist()))
valid_dataset = TensorDataset(torch.tensor(valid['id'].tolist()), 
                              torch.tensor(valid['mask'].tolist()), 
                              torch.tensor(valid['fraudulent'].tolist()))
test_dataset = TensorDataset(torch.tensor(test['id'].tolist()), 
                             torch.tensor(test['mask'].tolist()))

# DataLoader를 사용하여 데이터를 배치 단위로 로드
train_dataloader = DataLoader(train_dataset, batch_size=args["batch_size"], shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=args["batch_size"], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)


# **3. 모델**
BERT 모델을 정의하고, 사전 훈련된 가중치 로드하세요.

In [ ]:
# (11) BERT 모델 정의 및 사전 훈련된 가중치 로드
model = 

model = model.to(device)
print(model)

# **4. 학습**

In [ ]:
def train(train_dataloader, valid_dataloader, model, device, args):
    """
    주어진 데이터로 모델을 학습시키는 함수입니다.

    Args:
        train_dataloader (DataLoader): 학습 데이터를 제공하는 DataLoader 객체
        valid_dataloader (DataLoader): 검증 데이터를 제공하는 DataLoader 객체
        model (torch.nn.Module): 학습할 모델
        device (torch.device): 사용할 디바이스 (CPU 또는 GPU)
        args (dict): 학습 관련 인자들을 포함한 딕셔너리

    Returns:
        None
    """
    # (12-1) 옵티마이저와 스케줄러 초기화
    optimizer = 
    scheduler = 
    
    # (12-2) 모델의 그래디언트 초기화
    model.zero_grad()
    
    for epoch in range(args["epochs"]):
        # (12-3) 모델을 훈련 모드로 설정
        model.train()
        
        total_loss = 0  # 전체 손실 초기화
        total_accuracy = 0  # 전체 정확도 초기화
        
        print(f'Epoch {epoch + 1}/{args["epochs"]}')
        
        for step, batch in enumerate(tqdm(train_dataloader)):
            batch = tuple(index.to(device) for index in batch)
            ids, masks, labels = batch
            
            # (12-4) Forward pass
            outputs = 
            
            # (12-5) 손실 계산
            loss = 
            
            # 손실 누적
            total_loss += loss.item()
            
            # 정확도 계산
            pred = [torch.argmax(logit).cpu().detach().item() for logit in outputs.logits]
            true = [label for label in labels.cpu().numpy()]
            accuracy = accuracy_score(true, pred)
            total_accuracy += accuracy
            
            # (12-6) Backward pass
            loss.backward()
            
            # 그래디언트 클리핑
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            # (12-7) 옵티마이져를 이용한 파라미터 업데이트
            
            
            # (12-8) 스케줄러로 러닝 레이트 업데이트
            
            
            # (12-9) 모델의 그래디언트 초기화
            
        
        # 평균 손실과 정확도 계산
        avg_loss = total_loss / len(train_dataloader)
        avg_accuracy = total_accuracy / len(train_dataloader)
        
        # 모델 체크포인트 저장
        os.makedirs("results", exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model': model,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item,
            }, f'./results/model_state_dict_epoch_{epoch+1}.pth')
            
        # 현재 에포크의 손실과 정확도 출력
        print(f'CheckPoint : model_state_dict_epoch_{epoch+1}.pth')
        print(f'train_loss : {avg_loss}, train_acc : {avg_accuracy}\n')


if __name__ == "__main__":
    train(train_dataloader, valid_dataloader, model, device, args)

# **5. 평가**

In [ ]:
def test(test_dataloader, model, device):
    """
    모델의 테스트를 수행하는 함수입니다.

    Args:
        test_dataloader (DataLoader): 테스트 데이터를 제공하는 DataLoader 객체
        model (torch.nn.Module): 평가할 모델
        device (torch.device): 사용할 디바이스 (CPU 또는 GPU)

    Returns:
        preds (list): 각 입력 예시에 대한 모델의 예측 결과 리스트
    """
    # (13-1) 모델을 평가 모드로 설정
    
    preds = []  # 예측 결과를 저장할 리스트
    
    # 각 배치에 대해 예측 수행
    for batch in tqdm(test_dataloader):
        # (13-2) 배치의 데이터를 GPU로 이동
        batch = 
        # (13-3) 입력 ID와 마스크를 추출
        ids, masks = 
        
        # (13-4) 그래디언트 계산 비활성화
        with torch.no_grad():
            # (13-5) 모델에 입력을 전달하여 예측 수행
            output = 
            # (13-6) 예측 결과를 CPU로 이동하여 추출
            pred = 
            preds.append(pred)  # 예측 결과를 리스트에 추가
    
    return preds


if __name__ == "__main__":
    # 테스트 데이터로 예측 수행
    preds = test(test_dataloader, model, device)

In [ ]:
submit = pd.read_csv(args["submit_path"])
submit['fraudulent'] = preds
preds.to_csv('submission_p2.csv', index=False)

In [ ]:
submit

# **6. 추론(Inference)**

## **Gemini API 연결**

In [ ]:
import google.generativeai as genai

def gemini(text):
    # (14-1) API 키 설정
    GOOGLE_API_KEY = 
    
    # (14-2) Google Generative AI 구성
    

    # (14-3) GenerativeModel 인스턴스 생성
    model = 
    
    # (14-4) 입력 텍스트를 사용하여 콘텐츠 생성
    response = 
    
    # 생성된 텍스트 출력
    print(response.text)

    return response.text

## **텍스트 생성**

In [ ]:
# LLM에 입력될 문장
gemini_text = "Please create the following job very long posting phrase: 'We are looking for a dedicated and experienced software engineer to join our team. The ideal candidate will have strong problem-solving skills, extensive knowledge of programming languages, and experience in developing scalable applications. This position offers a competitive salary and a comprehensive benefits package."

In [ ]:
# LLM에 문장 입력
text = gemini(gemini_text)

## **텍스트 데이터 전처리**

In [ ]:
data = pd.DataFrame({'text': [text]})

In [ ]:
# 기본 데이터 전처리
## 영어 언어 모델 'en_core_web_sm'을 로드합니다.
nlp = spacy.load("en_core_web_sm")

## 텍스트 데이터 전처리를 수행합니다.
data['text'] = text_preprocessing(data['text'])

## 불용어를 제거합니다.
data['text'] = remove_stopwords(data['text'], nlp)

## 각 단어를 기본형으로 변환하여 텍스트를 정제합니다.
data['text'] = lemmatize_text(data['text'], nlp)

In [ ]:
# BERT 모델 입력을 위한 데이터 전처리
## 특수 토큰 추가 함수 호출
document = add_special_token(data['text'])

## 토큰화 함수 호출하여 텍스트를 토큰 ID로 변환
ids = tokenization(document)

## 패딩 함수 호출하여 모든 시퀀스의 길이를 일치시킴
ids = padding(ids, args)

## 어텐션 마스크 생성 함수 호출
masks = attention_mask(ids)

## 데이터프레임에 토큰 ID 및 어텐션 마스크 열 추가
data['id'] = ids
data['mask'] = masks

## 메모리에서 불필요한 변수 삭제
del document, ids, masks

In [ ]:
data

## **추론**

In [ ]:
def inference(data, model, device):
    """
    모델을 사용하여 단일 예제에 대해 추론을 수행하는 함수입니다.

    Args:
        data (dict): 추론할 단일 예제 데이터 딕셔너리
        model (torch.nn.Module): 평가할 모델
        device (torch.device): 사용할 디바이스

    Returns:
        int: 예측된 클래스 레이블
    """
    # (15-1) 모델을 평가 모드로 설정
    

    # (15-2) 데이터를 텐서로 변환하여 디바이스로 이동
    id = 
    mask = 

    # (15-3) 그래디언트 계산 비활성화
    
        # (15-4) 모델에 입력을 전달하여 예측 수행
        output = 
        pred = 

    return pred


if __name__ == "__main__":
    # 단일 예제에 대해 추론 수행
    predicted_label = inference(data, model, device)
    print(f"Job Posting : {data['text']}")
    print(f"Predicted Label : {predicted_label}")